In [ ]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm
import bz2
import sqlite3
import difflib
import gzip
import json
import re
import hashlib
from datetime import datetime
from datetime import timezone
import scipy.stats

In [ ]:
# load predictions
pred_dir = '/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/audit/predictions'

pred_0_30 = pd.read_pickle(os.path.join(pred_dir, 'GB_50_5_0_30_prediction_2020-08-01.pkl'))
pred_0_300 = pd.read_pickle(os.path.join(pred_dir, 'GB_50_5_0_300_prediction_2020-08-01.pkl'))

### Pairwise comparisons of time thresholds

In [ ]:
# 